In [1]:
import os
import numpy as np
import joblib                                                              #saving the fitted svm model/classifier
from sklearn import svm

In [2]:
emailspam=[]
directory="data/spam"                                                       #reading the spam emails
for file in sorted(os.listdir(directory)):
    f=open(directory+"/"+file,"r",encoding='cp437')
    b=f.readline().lower()
    b=b.replace("subject: fw :","")
    b=b.replace("subject: fw : fw :","")
    b=b.replace("subject: re : re :","")
    b=b.replace("subject: re :","")
    b=b.replace("subject: ","")
    a=b+f.read().lower()
    emailspam.append(a)
    
emailham=[]
directory="data/ham"                                                        #reading the ham emails
for file in sorted(os.listdir(directory)):
    f=open(directory+"/"+file,"r",encoding='cp437')
    a=f.readline().lower()
    start=1
    b=""
    while a!="":
        if start==1:
            b+=a
            a=f.readline()
        else:
            a=f.readline()
            
        if a[:7]=="subject":
            start=1
        elif a[:10]=="- - - - - " or a=="":
            start=0
            emailham.append(b)
            b=""

            
for i in range(len(emailham)):
    b=emailham[i]
    b=b.replace("subject: fw :","")
    b=b.replace("subject: fw : fw :","")
    b=b.replace("subject: re : re :","")
    b=b.replace("subject: re :","")
    b=b.replace("subject: ","")
    emailham[i]=b
    
s_len=len(emailspam)
h_len=len(emailham)
emails=emailspam+emailham
labels=np.hstack(((np.ones(s_len),np.zeros(h_len))))                           
emails=np.vstack((np.array(emails).transpose(),labels)).transpose()     #creating a list of emails with corresponding labels

In [3]:
def symbol_color_removal(email):                      #function to remove punctuation marks and  hex color codes
    e_mod=email
    sym=["\\","&","!",".",",","/","+","*","%","(",")",":",";","\"","_","?","=","@","{","}","[","]","<",">","|"]
    
    f=open("data/color_codes.txt","r")
    color=f.read().split("\n")
    for s in sym:
        e_mod=e_mod.replace(s," ")
    e_mod=e_mod.replace("\'","")
    e_mod=e_mod.replace("\n"," ")
    e_mod=e_mod.replace("-","")
    for c in color:
        e_mod=e_mod.replace(c,"")
        
    return e_mod

def lemma_dict():   #creating a python dictionary with keys as words and values as the corresponding root words(lemma)  
    l_dic={}
    f=open("data/lemma.txt","r")
    for i in range(20000):
        a=f.readline()
        b=a.replace("\n","")
        c=b.strip().split("->")
        d=c[1].strip().split(",")
        for ele in d:
            l_dic[ele]=c[0]
            
    return l_dic

def lemmatisation(email,dic):   #function to lemmatise the emails using the lemma dictionary( generated via lemma_dict())
    
    for key in dic.keys():
        e_mod=email.replace(key," "+dic[key]+" ")
    
    return e_mod


def num_token(email):                              #function to remove the digits
    num=["0","1","2","3","4","5","6","7","8","9"]
    e_mod=email
    for n in num:
        e_mod=e_mod.replace(n," ")
    
    return e_mod

def stop_words():                              #reading the English stop_words file
    f=open("data/stop_words.txt","r")
    stop_words=f.read().split("\n")

    return stop_words

def html_words():
    f=open("data/html_words.txt","r")          #reading the html_words file
    h_words=f.read().split("\n")

    return h_words

def rtf_words():                              #reading the rtf_words file
    f=open("data/rtf_words.txt","r")
    r_words=f.read().split("\n")

    return r_words

def idf(bag,lis):                             #function to compute the idf vector for the emails/documents
    idf_list=[]
    N=len(lis)
    for term in bag:
        a=0
        for doc in lis:
            if term in doc:
                a+=1
        idf_list.append(a)
    idf_vector=np.log(N/np.array(idf_list,dtype="float64"))
    return idf_vector

def tf(bag,doc):
    N=len(bag)
    tf_list=[]                                  #func to compute the term frequency vector for each email
    for i in range(N):
        count=doc.count(bag[i])
        tf_list.append(count/len(doc))
    tf_vector=np.array(tf_list,dtype="float64")
    return tf_vector

def tf_idf_vectorisation(bag,emails):
    idf_v=idf(bag,emails)
    N=len(bag)
    num_of_docs=len(emails)
    X=np.empty((N,num_of_docs))                #converts the list of emails into corresponding tf-idf vector array
    for i in range(num_of_docs):
        tf_v=tf(bag,emails[i])
        X[:,i]=tf_v*idf_v      
    
    return X

def smote(X,N,k=5):
    X_mod=X.transpose()                     # SMOTE for balancing the minority class(spams) 
    for i in range(X.shape[1]):
        current=X[:,i]
        dist=np.linalg.norm(X.transpose()-X[:,i],axis=1)
        dist_list=dist.tolist()
        neighbours=[]
        for j in range(k):
            min_pos=dist_list.index(min(dist_list))
            neighbours.append(X[:,min_pos])
            dist_list[min_pos]=max(dist_list)+1000

        a=np.arange(k)
        np.random.shuffle(a)
        neighbours_array=(np.array(neighbours))[a[:N]]
        c=np.random.uniform(0.0001,1,N)
        new_x=current+(c*(neighbours_array-current).transpose()).transpose()
        X_mod=np.vstack((X_mod,new_x))
        
    return (X_mod.transpose())

In [4]:
lemma_dictionary=lemma_dict()                       # generating the lemma dictionary
for i in range(len(emails)):
    emails[i,0]=symbol_color_removal(emails[i,0])       
    emails[i,0]=num_token(emails[i,0])
    emails[i,0]=lemmatisation(emails[i,0],lemma_dictionary)
    
s_words=stop_words()
h_words=html_words()
r_words=rtf_words()
length=len(emails)                   #processing such that each email is converted into list of words(filtered/parsed)
emails_list=[]
emails_list_labels=[]
for i in range(length):
    e=emails[i,0].strip().split(" ")   
    l=[x for x in e if x not in s_words and  x not in h_words and x not in r_words and x!=" " and x!="" and x!="#"]
    if l!=[]:
        emails_list.append(l)
        emails_list_labels.append(emails[i,1])                  
        
emails_list_flat=[i for j in emails_list for i in j]
emails_bag=list(set(emails_list_flat))      #creating the bag_of_words(unique words across the documents/list of emails)

X_nolabel=tf_idf_vectorisation(emails_bag,emails_list)   # performing tf-idf


pos=list(map(float,emails_list_labels)).index(0.0)
no_of_emails=len(emails_list_labels)
no_of_spams=pos
no_of_hams=no_of_emails-no_of_spams
rate=int(no_of_hams/no_of_spams) -1         #determining how many neighbours to consider per email or feature vector

spam_balanced=smote(X_nolabel[:,:no_of_spams],rate)     #performing smote

X=np.vstack((spam_balanced.transpose(),X_nolabel[:,no_of_spams:].transpose())).transpose()
spam_count_balanced=spam_balanced.shape[1]
y=np.hstack((np.ones(spam_count_balanced),np.zeros(no_of_hams)))

np.save("X.npy",X)               #saving the (feature extracted and balanced) dataset
np.save("y.npy",y)               #saving the corresponding labels
np.save("lemma_dictionary.npy",lemma_dictionary) # saving the lemma_dictionary generated(loaded in "prediction.ipynb" to lemmatise the test emails)
np.save("bagofwords.npy",emails_bag)        #saving the bag_of_words list(loaded in "prediction.ipynb" for tf-idf vectorisation)

lsvc=svm.LinearSVC(C=10,max_iter=10000)        
lsvc.fit(X.transpose(),y)                   #training / fitting the linear svm model     

joblib.dump(lsvc,"svm_classifier.sav") # saving the trained model/ classifier(loaded in "prediction.ipynb" to predict the test labels)

['svm_classifier.sav']